# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 18 2022 4:02PM,237590,18,6478359,"Blinc International, Inc.",Released
1,May 18 2022 3:57PM,237589,10,SO202201389,Yusen – 3D Matrix,Released
2,May 18 2022 3:54PM,237587,20,ALUR094936201,Alumier Labs Inc.,Released
3,May 18 2022 3:54PM,237587,20,ALUR187057763,Alumier Labs Inc.,Released
4,May 18 2022 3:54PM,237587,20,ALUR302275603,Alumier Labs Inc.,Released
5,May 18 2022 3:54PM,237587,20,ALUR574701359,Alumier Labs Inc.,Released
6,May 18 2022 3:54PM,237587,20,ALUR382799256,Alumier Labs Inc.,Released
7,May 18 2022 3:54PM,237587,20,ALUR396731585,Alumier Labs Inc.,Released
8,May 18 2022 3:54PM,237587,20,ALUR425535009,Alumier Labs Inc.,Released
9,May 18 2022 3:54PM,237587,20,ALUR577045938,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,237586,Released,22
45,237587,Released,35
46,237588,Released,3
47,237589,Released,1
48,237590,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237586,NaN,NaN,22.0
237587,NaN,NaN,35.0
237588,NaN,NaN,3.0
237589,NaN,NaN,1.0
237590,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237448,0.0,0.0,1.0
237478,0.0,0.0,45.0
237507,13.0,3.0,0.0
237510,2.0,1.0,1.0
237513,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237448,0,0,1
237478,0,0,45
237507,13,3,0
237510,2,1,1
237513,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237448,0,0,1
1,237478,0,0,45
2,237507,13,3,0
3,237510,2,1,1
4,237513,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237448,,,1
1,237478,,,45
2,237507,13,3,
3,237510,2,1,1
4,237513,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 18 2022 4:02PM,237590,18,"Blinc International, Inc."
1,May 18 2022 3:57PM,237589,10,Yusen – 3D Matrix
2,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.
37,May 18 2022 3:54PM,237586,20,Reliable 1 Laboratories LLC
59,May 18 2022 3:53PM,237588,10,"Herbivore Botanicals, LLC."
62,May 18 2022 3:52PM,237583,20,Reliable 1 Laboratories LLC
87,May 18 2022 3:51PM,237585,19,"GCH Granules USA, Inc."
88,May 18 2022 3:51PM,237573,10,Bio-PRF
108,May 18 2022 3:47PM,237584,10,Emerginnova
111,May 18 2022 3:33PM,237579,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 18 2022 4:02PM,237590,18,"Blinc International, Inc.",,,1
1,May 18 2022 3:57PM,237589,10,Yusen – 3D Matrix,,,1
2,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.,,,35
3,May 18 2022 3:54PM,237586,20,Reliable 1 Laboratories LLC,,,22
4,May 18 2022 3:53PM,237588,10,"Herbivore Botanicals, LLC.",,,3
5,May 18 2022 3:52PM,237583,20,Reliable 1 Laboratories LLC,,,25
6,May 18 2022 3:51PM,237585,19,"GCH Granules USA, Inc.",,,1
7,May 18 2022 3:51PM,237573,10,Bio-PRF,,,20
8,May 18 2022 3:47PM,237584,10,Emerginnova,,,3
9,May 18 2022 3:33PM,237579,10,ISDIN Corporation,,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 4:02PM,237590,18,"Blinc International, Inc.",1,,
1,May 18 2022 3:57PM,237589,10,Yusen – 3D Matrix,1,,
2,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.,35,,
3,May 18 2022 3:54PM,237586,20,Reliable 1 Laboratories LLC,22,,
4,May 18 2022 3:53PM,237588,10,"Herbivore Botanicals, LLC.",3,,
5,May 18 2022 3:52PM,237583,20,Reliable 1 Laboratories LLC,25,,
6,May 18 2022 3:51PM,237585,19,"GCH Granules USA, Inc.",1,,
7,May 18 2022 3:51PM,237573,10,Bio-PRF,20,,
8,May 18 2022 3:47PM,237584,10,Emerginnova,3,,
9,May 18 2022 3:33PM,237579,10,ISDIN Corporation,10,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 4:02PM,237590,18,"Blinc International, Inc.",1,,
1,May 18 2022 3:57PM,237589,10,Yusen – 3D Matrix,1,,
2,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.,35,,
3,May 18 2022 3:54PM,237586,20,Reliable 1 Laboratories LLC,22,,
4,May 18 2022 3:53PM,237588,10,"Herbivore Botanicals, LLC.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 4:02PM,237590,18,"Blinc International, Inc.",1.0,NaN,NaN
1,May 18 2022 3:57PM,237589,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.,35.0,NaN,NaN
3,May 18 2022 3:54PM,237586,20,Reliable 1 Laboratories LLC,22.0,NaN,NaN
4,May 18 2022 3:53PM,237588,10,"Herbivore Botanicals, LLC.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 4:02PM,237590,18,"Blinc International, Inc.",1.0,0.0,0.0
1,May 18 2022 3:57PM,237589,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,May 18 2022 3:54PM,237587,20,Alumier Labs Inc.,35.0,0.0,0.0
3,May 18 2022 3:54PM,237586,20,Reliable 1 Laboratories LLC,22.0,0.0,0.0
4,May 18 2022 3:53PM,237588,10,"Herbivore Botanicals, LLC.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4276018,232.0,17.0,2.0
12,237541,0.0,1.0,0.0
16,712651,11.0,13.0,0.0
18,1187678,7.0,3.0,13.0
19,712678,2.0,1.0,0.0
20,1662890,112.0,1.0,0.0
21,475151,2.0,0.0,0.0
22,475141,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4276018,232.0,17.0,2.0
1,12,237541,0.0,1.0,0.0
2,16,712651,11.0,13.0,0.0
3,18,1187678,7.0,3.0,13.0
4,19,712678,2.0,1.0,0.0
5,20,1662890,112.0,1.0,0.0
6,21,475151,2.0,0.0,0.0
7,22,475141,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,232.0,17.0,2.0
1,12,0.0,1.0,0.0
2,16,11.0,13.0,0.0
3,18,7.0,3.0,13.0
4,19,2.0,1.0,0.0
5,20,112.0,1.0,0.0
6,21,2.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,232.0
1,12,Released,0.0
2,16,Released,11.0
3,18,Released,7.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Completed,2.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
Executing,17.0,1.0,13.0,3.0,1.0,1.0,0.0,0.0
Released,232.0,0.0,11.0,7.0,2.0,112.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,17.0,1.0,13.0,3.0,1.0,1.0,0.0,0.0
2,Released,232.0,0.0,11.0,7.0,2.0,112.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,17.0,1.0,13.0,3.0,1.0,1.0,0.0,0.0
2,Released,232.0,0.0,11.0,7.0,2.0,112.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()